In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np

---

In [3]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 
              'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 
              'view':int}

In [4]:
train = pd.read_csv('./data/kc_house_data_small_train.csv', dtype=dtype_dict, index_col=0)
validation = pd.read_csv('./data/kc_house_data_validation.csv', dtype=dtype_dict, index_col=0)
test = pd.read_csv('./data/kc_house_data_small_test.csv', dtype=dtype_dict, index_col=0)

In [5]:
def get_numpy_data(df, features, output):
    df['constant'] = 1 # add a constant column to an DataFrame
    features = ['constant'] + features
    df_features = df.loc[:, features]
    features_matrix = df_features.values
    output_array = df.loc[:, output].values
    return features_matrix, output_array

In [6]:
def normalize_features(features_matrix):
    norms = np.linalg.norm(features_matrix, axis=0)
    normalized_features = features_matrix / norms
    return (normalized_features, norms)

In [7]:
feature_list = ['bedrooms',  
                'bathrooms',  
                'sqft_living',  
                'sqft_lot',  
                'floors',
                'waterfront',  
                'view',  
                'condition',  
                'grade',  
                'sqft_above',  
                'sqft_basement',
                'yr_built',  
                'yr_renovated',  
                'lat',  
                'long',  
                'sqft_living15',  
                'sqft_lot15']

In [8]:
X_Tr, y_Tr = get_numpy_data(train, feature_list, 'price')
X_Va, y_Va = get_numpy_data(validation, feature_list, 'price')
X_Te, y_Te = get_numpy_data(test, feature_list, 'price')

In computing distances, it is crucial to normalize features. Otherwise, for example, the ‘sqft_living’ feature (typically on the order of thousands) would exert a much larger influence on distance than the ‘bedrooms’ feature (typically on the order of ones). We divide each column of the training feature matrix by its 2-norm, so that the transformed column has unit norm.

IMPORTANT: Make sure to store the norms of the features in the training set. The features in the test and validation sets must be divided by these same norms, so that the training, test, and validation sets are normalized consistently.

In [9]:
features_train, norms = normalize_features(X_Tr)
features_valid = X_Va / norms
features_test = X_Te / norms

# Compute a single distance

To start, let's just explore computing the “distance” between two given houses. We will take our query house to be the first house of the test set and look at the distance between this house and the 10th house of the training set.

To see the features associated with the query house, print the first row (index 0) of the test feature matrix. You should get an 18-dimensional vector whose components are between 0 and 1. Similarly, print the 10th row (index 9) of the training feature matrix.

In [10]:
print(features_test[0])
print('')
print(features_train[9])

[ 0.01345102  0.01551285  0.01807473  0.01759212  0.00160518  0.017059
  0.          0.05102365  0.0116321   0.01564352  0.01362084  0.02481682
  0.01350306  0.          0.01345387 -0.01346922  0.01375926  0.0016225 ]

[ 0.01345102  0.01163464  0.00602491  0.0083488   0.00050756  0.01279425
  0.          0.          0.01938684  0.01390535  0.0096309   0.
  0.01302544  0.          0.01346821 -0.01346251  0.01195898  0.00156612]


Note: Do not use the ‘np.linalg.norm’ function; use ‘np.sqrt’, ‘np.sum’, and the power operator (**) instead. The latter approach is more easily adapted to computing multiple distances at once.

In [11]:
def euclidean_distance(feature1, feature2):
    return np.sqrt( np.sum( (feature1 - feature2)**2 ) )

In [12]:
euclidean_distance(features_test[0], features_train[9])

0.05972359371398078

# Compute multiple distances

Of course, to do nearest neighbor regression, we need to compute the distance between our query house and all houses in the training set.

To visualize this nearest-neighbor search, let's first compute the distance from our query house (features_test\[0\]) to the first 10 houses of the training set (features_train\[0:10\]) and then search for the nearest neighbor within this small set of houses. Through restricting ourselves to a small set of houses to begin with, we can visually scan the list of 10 distances to verify that our code for finding the nearest neighbor is working.

Write a loop to compute the Euclidean distance from the query house to each of the first 10 houses in the training set.

In [13]:
distance_first10 = np.array([euclidean_distance(features_test[0], features_train[i])
                             for i in range(10)])

In [14]:
print(distance_first10)

[0.06027471 0.08546881 0.06149946 0.05340274 0.05844484 0.05987922
 0.0546314  0.05543108 0.05238363 0.05972359]


In [15]:
print(np.argmin(distance_first10))

8


It is computationally inefficient to loop over computing distances to all houses in our training dataset. Fortunately, many of the numpy functions can be vectorized, applying the same operation over multiple values or vectors. We now walk through this process. (The material up to \#13 is specific to numpy; if you are using other languages such as R or Matlab, consult relevant manuals on vectorization.)

Consider the following loop that computes the element-wise difference between the features of the query house (features_test\[0\]) and the first 3 training houses (features_train\[0:3\]):

In [16]:
for i in range(3):
    print(features_train[i]-features_test[0])
    # should print 3 vectors of length 18

[ 0.00000000e+00 -3.87821276e-03 -1.20498190e-02 -1.05552733e-02
  2.08673616e-04 -8.52950206e-03  0.00000000e+00 -5.10236549e-02
  0.00000000e+00 -3.47633726e-03 -5.50336860e-03 -2.48168183e-02
 -1.63756198e-04  0.00000000e+00 -1.70254220e-05  1.29876855e-05
 -5.14364795e-03  6.69281453e-04]
[ 0.00000000e+00 -3.87821276e-03 -4.51868214e-03 -2.26610387e-03
  7.19763456e-04  0.00000000e+00  0.00000000e+00 -5.10236549e-02
  0.00000000e+00 -3.47633726e-03  1.30705004e-03 -1.45830788e-02
 -1.91048898e-04  6.65082271e-02  4.23090220e-05  6.16364736e-06
 -2.89330197e-03  1.47606982e-03]
[ 0.00000000e+00 -7.75642553e-03 -1.20498190e-02 -1.30002801e-02
  1.60518166e-03 -8.52950206e-03  0.00000000e+00 -5.10236549e-02
  0.00000000e+00 -5.21450589e-03 -8.32384500e-03 -2.48168183e-02
 -3.13866046e-04  0.00000000e+00  4.70885840e-05  1.56292487e-05
  3.72914476e-03  1.64764925e-03]


The subtraction operator (-) in numpy is vectorized as follows:

In [17]:
print(features_train[0:3] - features_test[0])

[[ 0.00000000e+00 -3.87821276e-03 -1.20498190e-02 -1.05552733e-02
   2.08673616e-04 -8.52950206e-03  0.00000000e+00 -5.10236549e-02
   0.00000000e+00 -3.47633726e-03 -5.50336860e-03 -2.48168183e-02
  -1.63756198e-04  0.00000000e+00 -1.70254220e-05  1.29876855e-05
  -5.14364795e-03  6.69281453e-04]
 [ 0.00000000e+00 -3.87821276e-03 -4.51868214e-03 -2.26610387e-03
   7.19763456e-04  0.00000000e+00  0.00000000e+00 -5.10236549e-02
   0.00000000e+00 -3.47633726e-03  1.30705004e-03 -1.45830788e-02
  -1.91048898e-04  6.65082271e-02  4.23090220e-05  6.16364736e-06
  -2.89330197e-03  1.47606982e-03]
 [ 0.00000000e+00 -7.75642553e-03 -1.20498190e-02 -1.30002801e-02
   1.60518166e-03 -8.52950206e-03  0.00000000e+00 -5.10236549e-02
   0.00000000e+00 -5.21450589e-03 -8.32384500e-03 -2.48168183e-02
  -3.13866046e-04  0.00000000e+00  4.70885840e-05  1.56292487e-05
   3.72914476e-03  1.64764925e-03]]


Note that the output of this vectorized operation is identical to that of the loop above, which can be verified below:

In [18]:
# verify that vectorization works
results = features_train[0:3] - features_test[0]
print( results[0] - (features_train[0]-features_test[0]) )
# should print all 0's if results[0] == (features_train[0]-features_test[0])
print( results[1] - (features_train[1]-features_test[0]) )
# should print all 0's if results[1] == (features_train[1]-features_test[0])
print( results[2] - (features_train[2]-features_test[0]) )
# should print all 0's if results[2] == (features_train[2]-features_test[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Aside: it is a good idea to write tests like this snippet whenever you are vectorizing a complicated operation.

# Perform 1-nearest neighbor regression

Now that we have the element-wise differences, it is not too hard to compute the Euclidean distances between our query house and all of the training houses. First, write a single-line expression to define a variable ‘diff’ such that ‘diff\[i\]’ gives the element-wise difference between the features of the query house and the i-th training house.

To test your code, print diff\[-1\].sum(), which should be -0.0934339605842.

In [19]:
diff = features_train - features_test[0]
print(diff[-1].sum())

-0.09343399874654643


The next step in computing the Euclidean distances is to take these feature-by-feature differences in ‘diff’, square each, and take the sum over feature indices. That is, compute the sum of squared feature differences for each training house (row in ‘diff’).

By default, ‘np.sum’ sums up everything in the matrix and returns a single number. To instead sum only over a row or column, we need to specifiy the ‘axis’ parameter described in the np.sum documentation. In particular, ‘axis=1’ computes the sum across each row.

So
```
np.sum(diff**2, axis=1)
```
computes this sum of squared feature differences for all training houses. Verify that the two expressions

In [20]:
np.sum(diff**2, axis=1)[15]

0.0033070590284564457

and

In [21]:
np.sum(diff[15]**2)

0.0033070590284564453

yield the same results. That is, the output for the 16th house in the training set is equivalent to having examined only the 16th row of ‘diff’ and computing the sum of squares on that row alone.

With this result in mind, write a single-line expression to compute the Euclidean distances from the query to all the instances. Assign the result to variable distances.

Hint: don't forget to take the square root of the sum of squares.

Hint: distances\[100\] should contain 0.0237082324496.

In [22]:
distances = np.sqrt( np.sum(diff**2, axis=1) )

In [23]:
distances[100]

0.023708232416678195

Now you are ready to write a function that computes the distances from a query house to all training houses. The function should take two parameters: (i) the matrix of training features and (ii) the single feature vector associated with the query.

In [24]:
def compute_distances(features_instances, features_query):
    diff = features_instances - features_query
    distances = np.sqrt( np.sum(diff**2, axis=1) )
    return distances

In [25]:
distances = compute_distances(features_train, features_test[2])
print(np.argmin(distances))

382


In [26]:
y_Tr[np.argmin(distances)]

249000.0

# Perform k-nearest neighbor regression

Using the functions above, implement a function that takes in

- the value of k;
- the feature matrix for the instances; and
- the feature of the query

and returns the indices of the k closest training houses. For instance, with 2-nearest neighbor, a return value of \[5, 10\] would indicate that the 6th and 11th training houses are closest to the query house.

In [27]:
def k_nearest_neighbors(k, feature_train, features_query):
    distances = compute_distances(feature_train, features_query)
    neighbors = np.argsort(distances)[:k]
    return neighbors

In [28]:
idx_4neighbors = k_nearest_neighbors(4, features_train, features_test[2])
print(idx_4neighbors)

[ 382 1149 4087 3142]


Now that we know how to find the k-nearest neighbors, write a function that predicts the value of a given query house. For simplicity, take the average of the prices of the k nearest neighbors in the training set. The function should have the following parameters:

- the value of k;
- the feature matrix for the instances;
- the output values (prices) of the instances; and
- the feature of the query, whose price we’re predicting.

The function should return a predicted value of the query house.

In [29]:
def predict_output_of_query(k, features_train, output_train, features_query):
    neighbors = k_nearest_neighbors(k, features_train, features_query)
    prediction = np.mean(output_train[neighbors])
    return prediction

In [30]:
predict_output_of_query(4, features_train, y_Tr, features_test[2])

413987.5

Finally, write a function to predict the value of each and every house in a query set. (The query set can be any subset of the dataset, be it the test set or validation set.) The idea is to have a loop where we take each house in the query set as the query house and make a prediction for that specific house. The new function should take the following parameters:

- the value of k;
- the feature matrix for the training set;
- the output values (prices) of the training houses; and
- the feature matrix for the query set.

The function should return a set of predicted values, one for each house in the query set.

In [31]:
def predict_output(k, features_train, output_train, features_query):
    predictions = np.array([predict_output_of_query(k, features_train, output_train, f_q)
                            for f_q in features_query])
    return predictions

In [32]:
yhat_test10_k10 = predict_output(10, features_train, y_Tr, features_test[:10])
print(yhat_test10_k10)

[881300.  431860.  460595.  430200.  766750.  667420.  350032.  512800.7
 484000.  457235. ]


In [33]:
print(np.argmin(yhat_test10_k10))
print(yhat_test10_k10[np.argmin(yhat_test10_k10)])

6
350032.0


# Choosing the best value of k using a validation set

There remains a question of choosing the value of k to use in making predictions. Here, we use a validation set to choose this value. Write a loop that does the following:

For k in \[1, 2, … 15\]:

- Make predictions for the VALIDATION data using the k-nearest neighbors from the TRAINING data.
- Compute the RSS on VALIDATION data

Report which k produced the lowest RSS on validation data.

In [34]:
RSS_valid = np.zeros(15)
ks = np.r_[1:16]

for i, k in enumerate(ks):
    yhat_valid = predict_output(k, features_train, y_Tr, features_valid)
    RSS_valid[i] = np.sum((yhat_valid - y_Va)**2)

In [35]:
idx_best_k = np.argmin(RSS_valid)
k_best = ks[idx_best_k]
print(k_best)

8


In [36]:
yhat_test = predict_output(k_best, features_train, y_Tr, features_test)
RSS_test = np.sum((yhat_test - y_Te)**2)
print('{:.4e}'.format(RSS_test))

1.3312e+14
